# Задание методы защиты в кредитно-финансовой сфере

## Таска
1. Необходимо провести анализ набора данных, рассказать, как он собран и для чего
2. Сформулировать бизнес-задачу (должна быть связана с выявлением мошенничества), указать целевую переменную, выбрать признаки
3. Подготовить наборы данных для применения методов машинного обучения 
4. Обучить модель. При этом необходимо использовать тот материал, которых мы проходили на индивидуальных семинарах. Не запрещается использовать и другие подходы, но по ним лучше дать теоретическую справку.
5. Подготовить презентацию и рассказать на семинаре результаты и проблемы, возникшие при моделировании.

### Данные
- [датасет тут](https://www.kaggle.com/datasets/nezukokamaado/auto-loan-dataset)

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("financial_loan.csv")
df.head()

,id,address_state,application_type,emp_length,emp_title,grade,home_ownership,issue_date,last_credit_pull_date,last_payment_date,...,sub_grade,term,verification_status,annual_income,dti,installment,int_rate,loan_amount,total_acc,total_payment
0,1077430,GA,INDIVIDUAL,< 1 year,Ryder,C,RENT,11-02-2021,13-09-2021,13-04-2021,...,C4,60 months,Source Verified,30000.0,0.0100,59.83,0.1527,2500,4,1009
1,1072053,CA,INDIVIDUAL,9 years,MKC Accounting,E,RENT,01-01-2021,14-12-2021,15-01-2021,...,E1,36 months,Source Verified,48000.0,0.0535,109.43,0.1864,3000,4,3939
2,1069243,CA,INDIVIDUAL,4 years,Chemat Technology Inc,C,RENT,05-01-2021,12-12-2021,09-01-2021,...,C5,36 months,Not Verified,50000.0,0.2088,421.65,0.1596,12000,11,3522
3,1041756,TX,INDIVIDUAL,< 1 year,barnes distribution,B,MORTGAGE,25-02-2021,12-12-2021,12-03-2021,...,B2,60 months,Source Verified,42000.0,0.0540,97.06,0.1065,4500,9,4911
4,1068350,IL,INDIVIDUAL,10+ years,J&J Steel Inc,A,MORTGAGE,01-01-2021,14-12-2021,15-01-2021,...,A1,36 months,Verified,83000.0,0.0231,106.53,0.0603,3500,28,3835


In [7]:
df.columns

Index(['id', 'address_state', 'application_type', 'emp_length', 'emp_title',
       'grade', 'home_ownership', 'issue_date', 'last_credit_pull_date',
       'last_payment_date', 'loan_status', 'next_payment_date', 'member_id',
       'purpose', 'sub_grade', 'term', 'verification_status', 'annual_income',
       'dti', 'installment', 'int_rate', 'loan_amount', 'total_acc',
       'total_payment'],
      dtype='object')

## Описание колонок
### Идентификаторы
- `id` - уникальный идентификатор кредитной заявки
- `member_id` - идентификатор заемщика
### Адрес и должность
- `adress_state` - штат проживания заемщика
- `emp_title` — должность / профессия
- `emp_length` — стаж работы (в годах, категориальный)
- `home_ownership` — тип владения жильем
### Параметры кредита
- `loan_amount` — сумма кредита
- `term` — срок кредита в месяцах
- `int_rate` — процентная ставка
- `installment` — ежемесячный платеж
- `purpose` — цель кредита
- `grade` — кредитный рейтинг (A–G)
- `sub_grade` — уточненный рейтинг (A1–G5)
### Финансовое состояние
- `annual_income` — годовой доход заемщика
- `dti` — доля долговых обязательств к доходу
- `total_acc` — общее количество кредитных счетов
### Статусы и даты
- `loan_status` — статус кредита (таргет)
- `issue_date` — дата выдачи кредита
- `last_payment_date` — дата последнего платежа
- `next_payment_date` — дата следующего платежа
- `last_credit_pull_date` — дата последней проверки кредитной истории
### Проверки и верификация
- `verification_status` — был ли доход подтвержден
- `application_type` — тип заявки (individual / joint)
### Результат по кредиту
- `total_payment` - общая сумма, выплаченная по кредиту
## Заметки
`loan_status` - целевая фича 
Может использоваться напрямую (дефолт / не дефолт) или как proxy для мошенничества. Есть поведенчские признаки, финансовые признаки, временные признаки (возможен лик)

# EDA начинается здесь
 - анализ данных
 - очистка датасета, подготовка к обучению

# 1. Анализ набора данных

## Как собран датасет
Датасет - исторические данные по автокредитам, выданным финансовым учреждением. \
Содержит информацию о:
- Заёмщиках (демография, доход, занятость, кредитная история)
- Условиях кредита (сумма, срок, процентная ставка, платежи)
- Истории обслуживания (даты платежей, текущий статус)

## Временной период
Данные охватывают 2021-2022 годы (формат ДД-ММ-ГГГГ).

## Источник данных
Датасет представляет собой обезличенные исторические данные кредитных заявок, опубликованные на Kaggle. Предположительно отражает данные реального кредитного учреждения.

## Цель сбора
- Отслеживание кредитного портфеля
- Анализ эффективности кредитной политики
- Мониторинг рисков и дефолтов

# 2. Бизнес-задача

Финансовые потери из-за кредитов, выданных заемщикам с искажённой или недостоверной информацией, что повышает вероятность дефолта.

#### Конкретная задача
Разработка модели для раннего выявления потенциально мошеннических заявок на этапе рассмотрения



# 3. EDA

In [2]:
print("Размер данных:", df.shape)

Размер данных: (38576, 24)


### Типы признаков

In [4]:
df.dtypes.reset_index().rename(columns={"index":"feature", 0:"dtype"})

,feature,dtype
0,id,int64
1,address_state,object
2,application_type,object
3,emp_length,object
4,emp_title,object
5,grade,object
6,home_ownership,object
7,issue_date,object
8,last_credit_pull_date,object
9,last_payment_date,object


### Количетсво пропусков и уникальных значений

In [5]:
info = pd.DataFrame({
    "dtype": df.dtypes.astype(str),
    "missing": df.isna().sum(),
    "missing_%": (df.isna().mean()*100).round(2),
    "n_unique": df.nunique(dropna=False),
}).sort_values("missing_%", ascending=False)

info

,dtype,missing,missing_%,n_unique
emp_title,object,1438,3.73,28526
id,int64,0,0.00,38576
purpose,object,0,0.00,14
total_acc,int64,0,0.00,82
loan_amount,int64,0,0.00,880
int_rate,float64,0,0.00,371
installment,float64,0,0.00,15132
dti,float64,0,0.00,2863
annual_income,float64,0,0.00,5096
verification_status,object,0,0.00,3


- пропуски почти отсутствуют, заметно только emp_title ~3.73%;
- emp_title имеет очень много уникальных значений (десятки тысяч). Его нельзя просто one-hot’нуть;
- application_type константный => можно смело удалять;
- если прогноз на момент выдачи, то убираем всё, что появляется после выдачи кредита (total_payment, last_payment_date, next_payment_date, last_credit_pull_dat);
- при временных данных нельзя ограничиваться train/test-разбиением, будем использовать out-of-time сплит;
- можно спокойно удалить следующие неинформативные столбцы: id, member_id.

### Анализ таргета

In [6]:
df["loan_status"].value_counts(dropna=False)

loan_status
Fully Paid     32145
Charged Off     5333
Current         1098
Name: count, dtype: int64

In [7]:
df_bin = df[df["loan_status"].isin(["Fully Paid", "Charged Off"])].copy()
df_bin["y"] = (df_bin["loan_status"] == "Charged Off").astype(int)
df_bin["y"].value_counts(normalize=True)

y
0    0.857703
1    0.142297
Name: proportion, dtype: float64

- у loan_status оставить только Fully Paid и Charged Off, а Current лучше исключить, потому что это «недонаблюдённый» статус;
- наблюдается дисбаланс классов: дефолтов ~14%;

# Обучение моделей
- baseline 
- анализ ошибок
- исправление ошибок

# План дальнейшего развития
- Выписать идеи по улучшению решения
- Выписать все, что не успели сделать в виду ограничений по времени